In [1]:
# Import libraries
import json
import numpy as np
import pandas as pd

In [2]:
# Select technology
TECH = 'B'

In [3]:
# Load settings for technology
with open(f"../data/tech{TECH}/settings.json") as sfile:
    settings = json.load(sfile)

In [4]:
# Load tech time series data
colnames = ["addr", "time", "r", "g"]
data = pd.read_csv(f"../data/tech{TECH}/relaxdata.tsv.gz", names=colnames, sep='\t')

In [5]:
# Check data
data.head()
data.tail()

,addr,time,r,g
42403246,6188,1.627208e+09,12763.561190,0.000078
42403247,6190,1.627208e+09,11411.352389,0.000088
42403248,6192,1.627208e+09,10819.457210,0.000092
42403249,6194,1.627208e+09,11125.209294,0.000090
42403250,6196,1.627208e+09,9866.573718,0.000101


In [6]:
# Subtract initial time and compute range and gi
data["time"] -= data.groupby("addr")["time"].transform("first") 
data["gi"] = data.groupby("addr")["g"].transform("first")
data["range"] = np.int32(data["gi"] / settings["gmax"] * 32)

# # For tech B, subtract and make new virtual cells
# if TECH == 'B':
#     data.drop(data[(data["time"] < 10)].index, inplace=True) # & (data["time"] > 1)
#     # data.loc[data["time"] >= 10, "addr"] += 1000000

#     # Do again
#     data["time"] -= data.groupby("addr")["time"].transform("first") 
#     data["gi"] = data.groupby("addr")["g"].transform("first")
#     data["range"] = np.int32(data["gi"] / settings["gmax"] * 32)

In [7]:
# Check data as needed
data.head()
# data.groupby('range')['range'].count() / 5500

,addr,time,r,g,gi,range
0,0,0.000000,394055.654398,0.000003,0.000003,0
1,0,0.008992,375526.174252,0.000003,0.000003,0
2,0,0.014989,349686.935004,0.000003,0.000003,0
3,0,0.017986,360976.997817,0.000003,0.000003,0
4,0,0.020987,356374.604909,0.000003,0.000003,0


In [8]:
# Get times
tdata = []
for time in settings["times"]:
    idx = (data["time"] - time).abs().groupby(data["addr"]).idxmin()
    d = data[data.index.isin(idx)]
    print(len(d))
    d = d[(d["time"] <= time*1.2) & (d["time"] >= time*0.8)]
    print(len(d))
    d["timept"] = time
    tdata.append(d)
tdata = pd.concat(tdata)

16384
16384
16384
16036
16384
16384
16384
16384
16384
10440
16384
16384
16384
16384
16384
16384


In [9]:
# Write to file
fopts = {"sep": '\t', "header": False, "index": False}
tdata.to_csv(f"../data/tech{TECH}/relaxdata.min.tsv.gz", **fopts)

# Write to modeling file
fopts["header"] = True
tdata[["addr","timept","g","range"]].sort_values(by="addr").to_csv(f"../data/tech{TECH}/model.tsv", **fopts)